In this tutorial we will create a CSP pipeline using MindPype.

In [1]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [2]:
import mindpype as mp

import numpy as np
from random import shuffle

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline. After creating the session we will create a graph which represents the pipeline.

In [3]:
# create a session and a graph
session = mp.Session.create()
trial_graph = mp.Graph.create(session)

CSP requires initialization data to use for training. Therefore, we will randomly generate values for the training data and labels and create tensors from these generated values using the ```create_from_data()``` factory method. 

In [4]:
# Create random initialization (training) data and labels
training_data = np.random.random((120,12,500))
labels = np.asarray([0]*60 + [1]*60)


# Create tensors from the data and labels
init_data = mp.Tensor.create_from_data(session,training_data)
init_labels = mp.Tensor.create_from_data(session,labels)


Next we will create our input and output data containers for the graph. Our pipeline will take input data and will output the predicted label so we will create a tensor object for our input and a scalar object for our output.

Then we will create virtual tensors to hold ant intermediate values that are calculated throughout the pipeline using the ```create_virtual()``` method. Since these intermediate values represent data that is only required in the proces of completing a calculation and we do not need to access them later, the virtual type is ideal. The virtual type provides temporary storage and enabled us to free up more memory.

In [5]:
# Create an input tensor with dummy data
input_tensor = mp.Tensor.create_from_data(session, np.random.randn(12, 500))

# Create a scalar that will be populated with the classifier label
classifier_label = mp.Scalar.create_from_value(session,-1)

# Create intermediate (virtual) tensors for the intermediate steps of the pipeline
intermediate_tensors = [mp.Tensor.create_virtual(session),
                        mp.Tensor.create_virtual(session),
                        mp.Tensor.create_virtual(session),
                        mp.Tensor.create_virtual(session)]

Next we will create a filter. We first set the parameter values for the filter, then we will use these values to create a butterworth filter using the ```create_butter()``` factory method.

In [6]:
# create filter parameters
order = 4
bandpass = (8,35) # in Hz
fs = 250

# Create a filter object using the parameters
filter_obj = mp.Filter.create_butter(session,order,bandpass,btype='bandpass',fs=fs,implementation='sos')

We will use an LDA classifier to predict the output labels, so we will create a classifier object using the ```create_LDA()``` factory method.

In [7]:
# Create a classifier object
classifier = mp.Classifier.create_LDA(session)

Now we will add the associated kernels to our graph using the ```add_to_graph()``` factory methods. Each kernel that we add to the graph represents a node that will execute a process. 

For our CSP pipeline, we will first filter the data. Then we will pass the filtered data (which is stored in a virtual tensor, intermediate_tensor[1]) to our CSP kernel which will calculate and apply our spatial filters to our data. Next, we will apply the variance and log kernels to the spatially filtered data (which is stored in a virtual tensor, intermediate_tensor[2]) to aid with feature extraction. Finally, we will use an LDA classifier to make our output label predictions.

In [8]:
# add the processing nodes to the graph using the factory methods
filter_kernel = mp.kernels.FilterKernel.add_to_graph(trial_graph,input_tensor,filter_obj,intermediate_tensors[0], axis = 1)

CSP_kernel = mp.kernels.CommonSpatialPatternKernel.add_to_graph(trial_graph, intermediate_tensors[0], intermediate_tensors[1], init_data, init_labels, 2)
var_kernel = mp.kernels.VarKernel.add_to_graph(trial_graph, intermediate_tensors[1], intermediate_tensors[2], axis = 1)
log_kernel = mp.kernels.LogKernel.add_to_graph(trial_graph, intermediate_tensors[2], intermediate_tensors[3])
LDA_kernel = mp.kernels.ClassifierKernel.add_to_graph(trial_graph, intermediate_tensors[3], classifier, classifier_label)

With all of our nodes added to our graph, we will then verify the graph using the ```verify()``` method. Verifying the graph orders the nodes for execution and ensure that the inputs and outputs of each processing node are appropriately typed and sized.

In [9]:
# verify the session (i.e. schedule the nodes) and ensure the inputs and outputs are connected properly
trial_graph.verify()

Verifying kernel Filter...
Verifying kernel CSP...
Verifying kernel Var...
Verifying kernel Log...
Verifying kernel Classifier...


After our graph has been verified, the next step is to initialize the graph. This step is required for pipelines that have methods that need to be trained or fit. For our pipeline the CSP and the LDA nodes require training/initialization so we will call the ```initialize()``` method on our graph.

In [10]:
# Since the graph contains nodes that must be initialzed/trained, we must call initialize() before running the graph
trial_graph.initialize()

We are now ready to run our pipeline. To run the graph for the provided input data, we use the ```execute()``` method.

In [11]:
# RUN!
trial_graph.execute()
# print the value of the most recent trial
print("Trial {}, Predicted label = {}\n".format(1, classifier_label.data))


print("Test Passed =D")

Executing trial with label: None
Trial 1, Predicted label = 1

Test Passed =D
